# Generating Poisson processes


In [ ]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd

## Regular Poisson process on rectangle

In [ ]:
def regular_on_rectangle(grid, random_component, x_lim, y_lim):
    dx = (x_lim[1] - x_lim[0])/grid[0]
    dy = (y_lim[1] - y_lim[0])/grid[1]
    mx = np.linspace(x_lim[0] + dx/2, x_lim[1] - dx/2, grid[0])
    my = np.linspace(y_lim[0] + dy/2, y_lim[1] - dy/2, grid[1])
    px,py=np.meshgrid(mx,my)
    rx = np.random.rand(grid[1],grid[0])* dx - dx/2
    ry = np.random.rand(grid[1],grid[0])* dy - dy/2
    px=px+random_component*rx
    py=py+random_component*ry
    p={'X':px.flatten(), 'Y':py.flatten()}
    points=pd.DataFrame(p)
    return points

### Generating exemplary process

In [ ]:
points=regular_on_rectangle([20,10],0.5,[0,20],[0,10])

### Visualisation

In [ ]:
fig, ax1=plt.subplots(figsize=(10,10))
sns.scatterplot(data=points,x="X",y="Y")
ax1.set_title("Regular Poisson process", fontsize=20)
plt.xlabel("X", fontsize=15)
plt.ylabel("Y", fontsize=15)
ax1.set_aspect('equal', 'box')

## Homogeneous Poisson process on rectangle

In [ ]:
def homogeneous_poisson_on_rectangle(intensity, x_lim, y_lim):
    a = (abs(x_lim[1]-x_lim[0]))*(abs(y_lim[1]-y_lim[0]))
    ex = intensity*a
    n = np.random.poisson(lam=ex, size=1)
    nn=n[0]
    x=np.random.rand(nn)*(x_lim[1]-x_lim[0])+x_lim[0]
    y=np.random.rand(nn)*(y_lim[1]-y_lim[0])+y_lim[0]
    p={"X":x.flatten(),"Y":y.flatten()}
    points=pd.DataFrame(p)
    return points

### Generating exemplary process

In [ ]:
points=homogeneous_poisson_on_rectangle(10,[0,20],[0,10])

### Visualisation

In [ ]:
fig, ax1=plt.subplots(figsize=(10,10))
sns.scatterplot(data=points,x="X",y="Y")
ax1.set_title("Homogeneous Poisson process", fontsize=20)
plt.xlabel("X", fontsize=15)
plt.ylabel("Y", fontsize=15)
ax1.set_aspect('equal', 'box')

## Unhomogeneous Poisson process on rectangle

In [ ]:
def unhomogeneous_poisson_on_rectangle(intensity_function, x_lim, y_lim):
    srx=(x_lim[0]+x_lim[1])/2
    sry=(y_lim[0]+y_lim[1])/2
    max_fun=sp.optimize.minimize(lambda x: -intensity_function(x), (srx, sry), bounds=((x_lim[0], x_lim[1]),(y_lim[0], y_lim[1])))
    points=homogeneous_poisson_on_rectangle(-max_fun.fun,x_lim,y_lim)
    points_probability=1-intensity_function([points["X"],points["Y"]])/-max_fun.fun
    a=np.zeros(points["X"].size)
    for i in range(points["X"].size):
        a[i]=np.random.choice([1,0],p=[points_probability[i],1-points_probability[i]])
    points2_a=np.where(a==0)
    points=points.loc[points2_a]
    return points

### Generating exemplary process

In [ ]:
points=unhomogeneous_poisson_on_rectangle(lambda x:x[0], [0,20], [0,10])

### Visualisation

In [ ]:
fig, ax1=plt.subplots(figsize=(10,10))
sns.scatterplot(data=points,x="X",y="Y")
ax1.set_title("Unhomogeneous Poisson process", fontsize=20)
plt.xlabel("X", fontsize=15)
plt.ylabel("Y", fontsize=15)
ax1.set_aspect('equal', 'box')